In [1]:
using DataFrames, MLDataUtils
using Clustering, Distances
using CSV
using Random
using Logging
using Printf


# Set up Logging - we recommend to use this command to avoid package warnings during the model training process.
logger = Logging.SimpleLogger(stderr, Logging.Warn);
global_logger(logger);


In [2]:
#### Set parameters for the learners
cr = :silhouette # VALIDATION criterion totrain ICOT
method = "ICOT_local"
warm_start = :oct; 
# in the paper,  awarmstart slution to intilazation algorithm, oct use kmeas labels as 
# as a uspervised algorithm. greedy option fits a cart tree to lebels? try? or not good as kmenas
geom_search = false 
# s a boolean parameter that controls where the algorithm will enable the geometric component of the feature space search. See details in Section 3.3.1 of the paper.
threshold = 0.99 # the percentile of gaps for each 
seed = 1
gridsearch = false
num_tree_restarts = 10
# is an integer specifying the number of random restarts to use in the local search algorithm. Must be positive and defaults to 100. The performance of the tree typically increases as this value is increased, but with quickly diminishing returns. The computational cost of training increases linearly with this value.
complexity_c = 0.0
min_bucket = 100


###### Step 1: Prepare the data
# Read the data - recommend the use of the (deprecated) readtable() command to avoid potential version conflicts with the CSV package.
dataset = CSV.File("/Users/rui/PycharmProjects/cluster_explain/dataset_noScaler.csv"); 
# println(dataset[1, :])


In [3]:
# data_x = select(dataset, Not(:label))
# data
#seperate x and y
dataset_df = DataFrame(dataset)
true_label = dataset[:, end]
data_x = dataset[:, 1:end-1]

# train model 

data_array = convert(Matrix{Float64}, data_x);
# Get the number of observations and features
n, p = size(data_array)
data_t = data_array';

##### Step 2: Fit K-means clustering on the dataset to generate a warm-start for ICOT
#Fix the seed
Random.seed!(seed);

# The ruspini dataset has pre-defined clusters, which we will use to select the cluster count (K) for the K-means algorithm. 
# In an unsupervised setting (with no prior-known K), the number of clusters for K means can be selected using the elbow method.
K = 10

# Run k-means and save the assignments 
kmeans_result = kmeans(data_t, K);
assignment = kmeans_result.assignments;

# concat data and feed into next model
data_full = DataFrame(hcat(dataset_df, assignment, makeunique=true));

#rename column
if !hasproperty(data_full, :true_labels)
    rename!(data_full, :label => :true_labels)
end

if !hasproperty(data_full, :kmean_assign)
    rename!(data_full, :x1 => :kmean_assign)
else
    println("all done")
end

,android/content/Context;getSystemService,java/net/URL;openConnection,android/os/PowerManager$WakeLock;release
,Int64,Int64,Int64
1,13,1,0
2,13,1,0
3,13,1,0
4,13,1,0
5,13,1,0
6,13,1,0
7,13,1,0
8,14,3,0
9,14,3,0


In [4]:
# Prepare data for ICOT: features are 
# stored in the matrix X, and the warm-start labels are stored in y
X = data_full[:, 1:end-2];
y = data_full[:, :true_labels];


maxdepth = 7# can change ! use grid search for tuning!


# Run ICOT with an OCT warm-start: fit an OCT as a supervised learning problem with labels "y" and use this as the warm-start
# function icot(maxdepth)
@time begin
warm_start = :oct
lnr_ws_oct = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
    minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
    geom_search=geom_search, geom_threshold=threshold)
run_time_icot_ls_oct = @elapsed ICOT.fit!(lnr_ws_oct, X, y)

score_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:dunnindex)
score_al_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:silhouette)
ICOT.showinbrowser(lnr_ws_oct)

@printf("dunnindex = %.4f\n", float(score_ws_oct))
@printf("silhouette = %.4f\n", float(score_al_ws_oct))

end
# end


┌ Warning: implicit `dims=2` argument now has to be passed explicitly to specify that distances between columns should be computed
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAIBase /Users/iai/.julia/packages/IAIBase/pOrUV/src/precompile.jl:19
Training trees...100%|██████████████████████████████████| Time: 0:00:58
Training trees...100%|██████████████████████████████████| Time: 0:01:29
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = dunn_score(::Array{Float64,2}, ::Array{Int64,1}) at clustering_tree.jl:132
└ @ ICOT /Users/iai/.julia/packages/ICOT/34UmY/src/clustering/clustering_tree.jl:132
Training trees...100%|██████████████████████████████████| Time: 0:00:48
Training trees...100%|██████████████████████████████████| Time: 0:01:15
Training trees...100%|██████████████████████████████████| Time: 0:00:47
Training trees...

dunnindex = 0.0822
silhouette = 0.5408
10411.731811 seconds (595.62 M allocations: 402.287 GiB, 1.87% gc time)


In [8]:

ICOT.IAI.showinbrowser(grid.lnr)

UndefVarError: [91mUndefVarError: grid not defined[39m

In [9]:
# Prepare data for ICOT: features are 
# stored in the matrix X, and the warm-start labels are stored in y
X = data_full[:, 1:end-2];
y = data_full[:, :true_labels];


maxdepth = 4 # can change ! use grid search for tuning!


# Run ICOT with an OCT warm-start: fit an OCT as a supervised learning problem with labels "y" and use this as the warm-start
# function icot(maxdepth)
@time begin
warm_start = :oct
lnr_ws_oct = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
    minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
    geom_search=geom_search, geom_threshold=threshold)
run_time_icot_ls_oct = @elapsed ICOT.fit!(lnr_ws_oct, X, y)

score_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:dunnindex)
score_al_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:silhouette)
ICOT.showinbrowser(lnr_ws_oct)

@printf("dunnindex = %.4f\n", float(score_ws_oct))
@printf("silhouette = %.4f\n", float(score_al_ws_oct))

end
# end


Training trees...100%|██████████████████████████████████| Time: 0:00:41
Training trees...100%|██████████████████████████████████| Time: 0:01:00
Training trees...100%|██████████████████████████████████| Time: 0:00:40
Training trees...100%|██████████████████████████████████| Time: 0:00:55
Training trees...100%|██████████████████████████████████| Time: 0:00:42
Training trees...100%|██████████████████████████████████| Time: 0:01:01
Training trees...100%|██████████████████████████████████| Time: 0:00:30
Training trees...100%|██████████████████████████████████| Time: 0:00:47
Training trees...100%|██████████████████████████████████| Time: 0:00:31
Training trees...100%|██████████████████████████████████| Time: 0:00:48
Training trees...100%|██████████████████████████████████| Time: 0:00:33
Training trees...100%|██████████████████████████████████| Time: 0:00:47
Training trees...100%|██████████████████████████████████| Time: 0:00:32
Training trees...100%|██████████████████████████████████| Time: 

dunnindex = 0.1950
silhouette = 0.4681
4742.371241 seconds (252.61 M allocations: 170.976 GiB, 1.37% gc time)


In [10]:
# Prepare data for ICOT: features are 
# stored in the matrix X, and the warm-start labels are stored in y
X = data_full[:, 1:end-2];
y = data_full[:, :true_labels];


maxdepth = 5 # can change ! use grid search for tuning!


# Run ICOT with an OCT warm-start: fit an OCT as a supervised learning problem with labels "y" and use this as the warm-start
# function icot(maxdepth)
@time begin
warm_start = :oct
lnr_ws_oct = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
    minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
    geom_search=geom_search, geom_threshold=threshold)
run_time_icot_ls_oct = @elapsed ICOT.fit!(lnr_ws_oct, X, y)

score_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:dunnindex)
score_al_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:silhouette)
ICOT.showinbrowser(lnr_ws_oct)

@printf("dunnindex = %.4f\n", float(score_ws_oct))
@printf("silhouette = %.4f\n", float(score_al_ws_oct))

end
# end

Training trees...100%|██████████████████████████████████| Time: 0:00:36
Training trees...100%|██████████████████████████████████| Time: 0:00:54
Training trees...100%|██████████████████████████████████| Time: 0:00:37
Training trees...100%|██████████████████████████████████| Time: 0:00:57
Training trees...100%|██████████████████████████████████| Time: 0:00:35
Training trees...100%|██████████████████████████████████| Time: 0:00:55
Training trees...100%|██████████████████████████████████| Time: 0:00:35
Training trees...100%|██████████████████████████████████| Time: 0:01:00
Training trees...100%|██████████████████████████████████| Time: 0:00:37
Training trees...100%|██████████████████████████████████| Time: 0:01:01
Training trees...100%|██████████████████████████████████| Time: 0:00:50
Training trees...100%|██████████████████████████████████| Time: 0:01:21
Training trees...100%|██████████████████████████████████| Time: 0:00:53
Training trees...100%|██████████████████████████████████| Time: 

dunnindex = 0.0103
silhouette = 0.5058
6468.486663 seconds (336.30 M allocations: 246.866 GiB, 0.78% gc time)
